# DLS Final - data cleaning
This notebook serves to clean the dataframe to include only schools that meet the following criteria
- Have at least 100 students taking the math test
- Is a California high school
- Has test scores for math

In [1]:
import pandas as pd
import os
import glob
import numpy as np
import shutil

In [2]:
data_w_scores = pd.read_csv("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\US_school_name_location\\Public_schools_merged_scores.csv")
data_overall = pd.read_csv("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\US_school_name_location\\Public_Schools.csv")


### Merge the school w scores dataframe with original dataframe to get the high school column from the original dataframe

In [3]:
data_merged_w_scores = pd.merge(data_w_scores, data_overall, on = 'NCESID')
data_merged_w_scores.columns

Index(['School_ID', 'X_x', 'Y_x', 'NCESID', 'NAME_x', 'ADDRESS_x', 'CITY_x',
       'STATE_x', 'ZIP_x', 'POPULATION_x', 'COUNTY_x', 'LATITUDE_x',
       'LONGITUDE_x', 'NAICS_DESC_x', 'Type', 'LEANM', 'SCHNAM',
       'ALL_MTH00NUMVALID_1415', 'ALL_MTH00PCTPROF_1415', 'X_y', 'Y_y',
       'OBJECTID', 'NAME_y', 'ADDRESS_y', 'CITY_y', 'STATE_y', 'ZIP_y', 'ZIP4',
       'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION_y', 'COUNTY_y', 'COUNTYFIPS',
       'COUNTRY', 'LATITUDE_y', 'LONGITUDE_y', 'NAICS_CODE', 'NAICS_DESC_y',
       'SOURCE', 'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'LEVEL_',
       'ENROLLMENT', 'ST_GRADE', 'END_GRADE', 'DISTRICTID', 'FT_TEACHER',
       'SHELTER_ID'],
      dtype='object')

In [4]:
data_merged_w_scores

,School_ID,X_x,Y_x,NCESID,NAME_x,ADDRESS_x,CITY_x,STATE_x,ZIP_x,POPULATION_x,...,VAL_METHOD,VAL_DATE,WEBSITE,LEVEL_,ENROLLMENT,ST_GRADE,END_GRADE,DISTRICTID,FT_TEACHER,SHELTER_ID
0,1,-1.645602e+07,9565275.267,20086200453,RAVEN SCHOOL,4762 OLD AIRPORT WAY,FAIRBANKS,AK,99709,1527,...,IMAGERY/OTHER,2011/10/14 00:00:00,http://www.ravenschool.com/,OTHER,1515,PK,12,200862,12,NOT AVAILABLE
1,2,-1.730828e+07,9539200.777,20086200386,MERRELINE A KANGAS SCHOOL,1ST & HILLTOP,RUBY,AK,99768,35,...,GEOCODE,2010/09/16 00:00:00,http://www.yksd.com/site/default.aspx?PageID=1,OTHER,31,PK,12,200862,4,NOT AVAILABLE
2,3,-1.760094e+07,9534463.580,20086200385,ANDREW K. DEMOSKI SCHOOL,123 FRONT ST,NULATO,AK,99765,52,...,IMAGERY/OTHER,2010/09/16 00:00:00,http://www.yksd.com/site/default.aspx?PageID=1,OTHER,47,PK,12,200862,5,NOT AVAILABLE
3,4,-1.662457e+07,9648059.672,20086200384,MINTO SCHOOL,LAKER ONE ST,MINTO,AK,99758,63,...,GEOCODE,2010/09/16 00:00:00,http://www.yksd.com/site/default.aspx?PageID=1,OTHER,57,PK,12,200862,6,NOT AVAILABLE
4,5,-1.676709e+07,9610260.064,20086200383,GLADYS DART SCHOOL,ELLIOTT HWY 162,MANLEY HOT SPRINGS,AK,99756,15,...,GEOCODE,2010/09/16 00:00:00,http://www.yksd.com/site/default.aspx?PageID=1,OTHER,13,PK,12,200862,2,NOT AVAILABLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15856,15857,-8.127420e+06,5062005.393,90531001611,BEECHER ROAD SCHOOL,40 BEECHER RD.,WOODBRIDGE,CT,6525,897,...,IMAGERY/OTHER,2015/06/26 00:00:00,http://www.woodbridge.k12.ct.us,ELEMENTARY,830,PK,06,905310,67,NOT AVAILABLE
15857,15858,-8.010066e+06,5150094.041,90537000573,WOODSTOCK MIDDLE SCHOOL,147 B-ROUTE 169,WOODSTOCK,CT,6281,434,...,IMAGERY/OTHER,2016/03/09 00:00:00,http://www.woodstockschools.net,MIDDLE,404,05,08,905370,30,NOT AVAILABLE
15858,15859,-8.010850e+06,5152676.672,90537001110,WOODSTOCK ELEMENTARY SCHOOL,24 FROG POND RD.,WOODSTOCK,CT,6281,477,...,IMAGERY/OTHER,2016/03/09 00:00:00,http://www.woodstockschools.net,ELEMENTARY,446,PK,04,905370,31,NOT AVAILABLE
15859,15860,-8.024214e+06,5091918.927,90537101221,NORWICH FREE ACADEMY,305 BROADWAY,NORWICH,CT,6360,2481,...,IMAGERY,2011/10/24 00:00:00,http://www.nfaschool.org,HIGH,2304,09,12,905371,177,10782473


### Remove the '/' from school names and replace with ' '

In [5]:
### Change name of schools that have a '/' in their name to a ' '
school_names = data_merged_w_scores['NAME_x']

bad_names_list = []
for a in school_names:
  try:
    a.split('/')[1]
    bad_names_list.append(a)
  except IndexError:
    pass

for a in bad_names_list:
  data_merged_w_scores['NAME_x'][data_merged_w_scores['NAME_x'] == a] = a.replace('/',' ')

C:\Users\lvden\AppData\Local\Temp\ipykernel_15736\3164372917.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merged_w_scores['NAME_x'][data_merged_w_scores['NAME_x'] == a] = a.replace('/',' ')
C:\Users\lvden\AppData\Local\Temp\ipykernel_15736\3164372917.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merged_w_scores['NAME_x'][data_merged_w_scores['NAME_x'] == a] = a.replace('/',' ')
C:\Users\lvden\AppData\Local\Temp\ipykernel_15736\3164372917.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

### Only select California high schools with at least 100 test takers 

In [6]:
data_merged_w_scores1 = data_merged_w_scores[data_merged_w_scores['STATE_x'] == 'CA'].reset_index(drop=True)

In [7]:
data_merged_w_scores2 = data_merged_w_scores1[data_merged_w_scores1['LEVEL_']=='HIGH'].reset_index(drop=True)

In [9]:
print(len(data_merged_w_scores2))

1288


In [13]:
for a in range(1,1000):
    print(str(a) +': '+ str(len(data_merged_w_scores2[data_merged_w_scores2['ALL_MTH00NUMVALID_1415']>=a])))

1: 1288
2: 1281
3: 1279
4: 1277
5: 1276
6: 1275
7: 1273
8: 1272
9: 1272
10: 1271
11: 1266
12: 1262
13: 1259
14: 1258
15: 1257
16: 1254
17: 1251
18: 1247
19: 1245
20: 1241
21: 1239
22: 1238
23: 1236
24: 1235
25: 1234
26: 1232
27: 1228
28: 1226
29: 1223
30: 1221
31: 1218
32: 1215
33: 1213
34: 1210
35: 1204
36: 1201
37: 1198
38: 1198
39: 1193
40: 1190
41: 1187
42: 1183
43: 1182
44: 1181
45: 1177
46: 1174
47: 1173
48: 1170
49: 1167
50: 1166
51: 1165
52: 1161
53: 1157
54: 1155
55: 1154
56: 1150
57: 1148
58: 1147
59: 1145
60: 1143
61: 1142
62: 1138
63: 1134
64: 1131
65: 1130
66: 1128
67: 1127
68: 1122
69: 1120
70: 1119
71: 1114
72: 1109
73: 1108
74: 1104
75: 1102
76: 1099
77: 1098
78: 1093
79: 1091
80: 1090
81: 1086
82: 1084
83: 1081
84: 1077
85: 1075
86: 1073
87: 1073
88: 1071
89: 1067
90: 1062
91: 1060
92: 1057
93: 1054
94: 1051
95: 1045
96: 1043
97: 1040
98: 1038
99: 1036
100: 1033
101: 1032
102: 1031
103: 1028
104: 1027
105: 1027
106: 1024
107: 1022
108: 1019
109: 1014
110: 1009
111: 100

In [17]:
data_merged_w_scores3 = data_merged_w_scores2[data_merged_w_scores2['ALL_MTH00NUMVALID_1415']>=100].reset_index(drop=True)

### Clean the percentage passing column by removing letter values and then taking the mean of range values

In [20]:
prof_vals = np.unique(data_merged_w_scores3['ALL_MTH00PCTPROF_1415'].values)
prof_vals

array(['10', '11', '12', '13', '14', '15', '15-19', '16', '17', '18',
       '19', '2', '20', '20-24', '21', '22', '23', '24', '25', '25-29',
       '26', '27', '28', '29', '3', '30', '30-34', '31', '32', '33', '34',
       '35', '35-39', '36', '37', '38', '39', '4', '40', '40-44', '41',
       '42', '43', '44', '45', '45-49', '45086', '45213', '46', '47',
       '48', '49', '5', '50', '50-54', '51', '52', '53', '54', '55',
       '55-59', '56', '57', '58', '59', '6', '60', '60-64', '61', '62',
       '63', '64', '65', '65-69', '66', '67', '68', '69', '7', '70',
       '70-74', '71', '72', '73', '74', '75', '75-79', '76', '77', '78',
       '8', '80', '80-84', '82', '83', '84', '85', '86', '88', '9', '90',
       '90-94', '91', '93', '96', '97', 'LE1', 'LE5'], dtype=object)

In [30]:
dirty_to_clean_dict = {}
for val in prof_vals:
    try:
        int(val)
        if int(val) < 100:
            dirty_to_clean_dict[val]=int(val)
        else:
            dirty_to_clean_dict[val]=999
    except ValueError:
        try:
            middle = int(val.split('-')[0]) + (int(val.split('-')[1]) - int(val.split('-')[0]))/2
            dirty_to_clean_dict[val] = middle
        except (IndexError, ValueError):
            dirty_to_clean_dict[val] = 999

In [32]:
dirty_to_clean_dict

{'10': 10,
 '11': 11,
 '12': 12,
 '13': 13,
 '14': 14,
 '15': 15,
 '15-19': 17.0,
 '16': 16,
 '17': 17,
 '18': 18,
 '19': 19,
 '2': 2,
 '20': 20,
 '20-24': 22.0,
 '21': 21,
 '22': 22,
 '23': 23,
 '24': 24,
 '25': 25,
 '25-29': 27.0,
 '26': 26,
 '27': 27,
 '28': 28,
 '29': 29,
 '3': 3,
 '30': 30,
 '30-34': 32.0,
 '31': 31,
 '32': 32,
 '33': 33,
 '34': 34,
 '35': 35,
 '35-39': 37.0,
 '36': 36,
 '37': 37,
 '38': 38,
 '39': 39,
 '4': 4,
 '40': 40,
 '40-44': 42.0,
 '41': 41,
 '42': 42,
 '43': 43,
 '44': 44,
 '45': 45,
 '45-49': 47.0,
 '45086': 999,
 '45213': 999,
 '46': 46,
 '47': 47,
 '48': 48,
 '49': 49,
 '5': 5,
 '50': 50,
 '50-54': 52.0,
 '51': 51,
 '52': 52,
 '53': 53,
 '54': 54,
 '55': 55,
 '55-59': 57.0,
 '56': 56,
 '57': 57,
 '58': 58,
 '59': 59,
 '6': 6,
 '60': 60,
 '60-64': 62.0,
 '61': 61,
 '62': 62,
 '63': 63,
 '64': 64,
 '65': 65,
 '65-69': 67.0,
 '66': 66,
 '67': 67,
 '68': 68,
 '69': 69,
 '7': 7,
 '70': 70,
 '70-74': 72.0,
 '71': 71,
 '72': 72,
 '73': 73,
 '74': 74,
 '75': 75

IndexError: list index out of range

In [78]:
# dirty_to_clean_dict = {'15-19':17, '20-24':22, '20-29':24.5, '21-39':30, '25-29':27, '30-34':32,
#                        '30-39':34.5, '35-39':37, '40-44':42, '40-49':44.5, '40-59':49.5, 
#                       '45-49':47, '45086':999, '45213':999, '45249':999, '50-54':52, '50-59':54.5,
#                       '55-59':57, '60-64':62, '60-79': 69.5, '65-69':67, '70-74':72, '70-79':74.5,
#                       '80-89':84.5, 'GE50':999, 'LE10':999, 'LE20':999, 'LE5':999, 'LT50':999,
#                        'PS':999}

In [33]:
### Input the newly cleaned score values to the dataframe
i=0
while i < len(data_merged_w_scores3):
    data_merged_w_scores3['ALL_MTH00PCTPROF_1415'].iloc[i] = dirty_to_clean_dict[data_merged_w_scores3['ALL_MTH00PCTPROF_1415'].iloc[i]]
    i+=1

C:\Users\lvden\AppData\Local\Temp\ipykernel_15736\2450398136.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merged_w_scores3['ALL_MTH00PCTPROF_1415'].iloc[i] = dirty_to_clean_dict[data_merged_w_scores3['ALL_MTH00PCTPROF_1415'].iloc[i]]


In [34]:
data_merged_w_scores4 = data_merged_w_scores3[data_merged_w_scores3['ALL_MTH00PCTPROF_1415'] != 999]

In [35]:
data_merged_w_scores4 = data_merged_w_scores4.reset_index(drop=True)
data_merged_w_scores4

,School_ID,X_x,Y_x,NCESID,NAME_x,ADDRESS_x,CITY_x,STATE_x,ZIP_x,POPULATION_x,...,VAL_METHOD,VAL_DATE,WEBSITE,LEVEL_,ENROLLMENT,ST_GRADE,END_GRADE,DISTRICTID,FT_TEACHER,SHELTER_ID
0,4720,-13036883.24,3883063.668,63432002688,SCRIPPS RANCH HIGH,10410 TREENA ST.,SAN DIEGO,CA,92131,2314,...,IMAGERY,2016/02/23 00:00:00,http://srhs.falcons.org,HIGH,2234,09,12,634320,80,NOT AVAILABLE
1,4721,-13044882.00,3882403.165,63432003939,CHARTER SCHOOL OF SAN DIEGO,10170 HUENNEKENS ST.,SAN DIEGO,CA,92121,1484,...,IMAGERY,2011/10/21 00:00:00,http://www.charterschool-sandiego.net,HIGH,1436,07,12,634320,48,NOT AVAILABLE
2,4748,-13047057.20,3870031.796,63432005434,CLAIREMONT HIGH,4150 UTE DR.,SAN DIEGO,CA,92117,965,...,IMAGERY,2011/10/21 00:00:00,http://sandi.net/clairemont,HIGH,922,09,12,634320,43,NOT AVAILABLE
3,4778,-13030110.55,3868410.812,63432005479,HENRY HIGH,6702 WANDERMERE DR.,SAN DIEGO,CA,92120,2492,...,IMAGERY,2011/10/21 00:00:00,NOT AVAILABLE,HIGH,2397,09,12,634320,95,10801177
4,4781,-13035289.07,3862971.745,63432005482,HOOVER HIGH,4474 EL CAJON BLVD.,SAN DIEGO,CA,92115,2278,...,IMAGERY,2010/05/20 00:00:00,http://www.hoovercardinals.org,HIGH,2187,09,12,634320,91,NOT AVAILABLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,13150,-13063093.30,4086697.511,60001409072,HESPERIA HIGH,9898 MAPLE AVE.,HESPERIA,CA,92345,2015,...,IMAGERY,2011/10/20 00:00:00,NOT AVAILABLE,HIGH,1923,09,12,600014,92,10819001
959,13152,-13056291.14,4084048.450,60001405093,SULTANA HIGH,17311 SULTANA AVE.,HESPERIA,CA,92345,1986,...,IMAGERY,2016/02/22 00:00:00,NOT AVAILABLE,HIGH,1901,09,12,600014,85,10801155
960,13156,-13502914.41,4694836.804,60001312390,WESTERN SIERRA COLLEGIATE ACADEMY,660 MENLO DR.,ROCKLIN,CA,95765,806,...,GEOCODE,2018/01/08 00:00:00,http://www.wscacademy.org,HIGH,770,07,12,600013,36,NOT AVAILABLE
961,13159,-13501874.65,4698261.746,60001311010,WHITNEY HIGH,701 WILDCAT BLVD.,ROCKLIN,CA,95765,1948,...,IMAGERY/OTHER,2011/10/19 00:00:00,http://www.rocklin.k12.ca.us,HIGH,1866,09,12,600013,82,NOT AVAILABLE


In [36]:
ncesid_df_list = data_merged_w_scores4['NCESID'].values
ncesid_df_list.sort()

In [37]:
ncesid_df_list = list(ncesid_df_list)

### Copy over only the images we have for these schools. Also need to check that the number of images is equal to the number of schools

In [38]:
small = glob.glob("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\img_data-20231201T222346Z-001\\img_data_screenshots\\small\\*")
print(len(small))

large = glob.glob("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\img_data-20231201T222346Z-001\\img_data_screenshots\\large\\*")
print(len(large))

1222
1222


In [39]:
### For some reason I decided to add a counter to the names. Dumb idea and here it is removed in the 
### names
ncesid_img_list = []
for path in small:
    val = path.split('_')[-1].split('.')[0]
    ncesid_img_list.append(int(val[0:11]))

In [40]:
ncesid_img_list[0:5]

[60000103278, 60000113829, 60000908779, 60001109065, 60001302405]

In [41]:
crossover_ncesid_list = []
for a in ncesid_img_list:
    if a in ncesid_df_list:
        crossover_ncesid_list.append(a)

In [42]:
len(crossover_ncesid_list)

906

In [43]:
### Copying the small files to a new location based on ncesid

i=0
for img in small:
    if int(img.split('_')[-1].split('.')[0][0:11]) in crossover_ncesid_list:
        new_path_base = "C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\img_data-20231201T222346Z-001\\img_data_screenshots_cali_high_100\\small\\"      
        original_name = img.split('_')[-1].split('.')[0][0:11]
        dest = new_path_base+ 'small_' + original_name + '.jpg'
        shutil.copy(img, dest)
        


In [44]:
### Copying the large files to a new location based on ncesid

i=0
for img in large:
    if int(img.split('_')[-1].split('.')[0][0:11]) in crossover_ncesid_list:
        new_path_base = "C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\img_data-20231201T222346Z-001\\img_data_screenshots_cali_high_100\\large\\"      
        original_name = img.split('_')[-1].split('.')[0][0:11]
        dest = new_path_base+ 'large_' + original_name + '.jpg'
        shutil.copy(img, dest)
        


In [45]:
data_merged_w_scores4.to_csv('cleaned_school_subset.csv')